In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # seaborn package for visualising
import plotly.express as px # plotly visualisation
import time
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
'''
List of files present
/kaggle/input/novel-corona-virus-2019-dataset/2019_nCoV_data.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_2019_ncov_recovered.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_2019_ncov_deaths.csv
/kaggle/input/novel-corona-virus-2019-dataset/time_series_2019_ncov_confirmed.csv
'''
#whole_data_path='../input/novel-corona-virus-2019-dataset/2019_nCoV_data.csv'
whole_data_path='../input/novel-corona-virus-2019-dataset/covid_19_data.csv'

#recovered_data_path='../input/novel-corona-virus-2019-dataset/time_series_2019_ncov_recovered.csv'
#deaths_data_path='../input/novel-corona-virus-2019-dataset/time_series_2019_ncov_deaths.csv'
#confirmed_data_path='../input/novel-corona-virus-2019-dataset/time_series_2019_ncov_confirmed.csv'

corona_data= pd.read_csv(whole_data_path)
#recovered_data=pd.read_csv(recovered_data_path)
#deaths_data=pd.read_csv(recovered_data_path)
#confirmed_data=pd.read_csv(recovered_data_path)



## Initial Dataset Description

Reading data from the datasets.Here is the description I have got from the dataset 
Dataset Link : [Corona Virus Dataset](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset)

    corona_data    --> Daily level information on the number of 2019-nCoV affected cases across the globe
    confirmed_data  --> Time series data of confirmed cases
    deaths_data    --> Time series data of death cases
    recovered_data --> Time series data of recovered cases

I thank John Hopkins University and SRK for providing the dataset.I hope to learn a lot from it .

### Part 1 : Corona Data

I have started with the corona data which I think is the main dataset from which other datasets have been created from. 

In [ ]:
corona_data.shape
#Total no. of rows

In [ ]:
corona_data.info()

 1. Current details of column and their data types

        | Column_Name         |Data_type|
        |:-------------------:|:-------:|
        |  Sno                | int64   |
        |   ObservationDate   | object  |
        |   Province/State    | object  |
        |   Country           | object  |
        |   Last Update       | object  |
        |   Confirmed         | float64 |
        |   Deaths            | float64 |
        |   Recovered         | float64 |
Removing the Last Update column as it is not currently needed.

In [ ]:
if 'Last Update' in corona_data.columns :
    corona_data=corona_data.drop('Last Update',axis=1)
elif  'SNo' in corona_data.columns:
    corona_data=corona_data.drop('SNo',axis=1)
    
#corona_data["Date"] = corona_data['ObservationDate'].astype('datetime64')
corona_data["ObservationDate"] = corona_data['ObservationDate'].astype('datetime64')

corona_data["Confirmed"] = corona_data['Confirmed'].astype('int64')
corona_data["Deaths"] = corona_data['Deaths'].astype('int64')
corona_data["Recovered"] = corona_data['Recovered'].astype('int64')

print('Minimum date collected - ',min(corona_data["ObservationDate"]))
print('Maximum date collected(Latest data can be retrieved now) - ',max(corona_data["ObservationDate"]))

corona_data=corona_data.rename(columns={"ObservationDate": "Date","Country/Region":"Country"})

We have converted the date from object format. The data is present from minimum of 22 Jan to 13 Feb 2020.This dataset is interesting because this dataset is for span of approx. 21 days and the virus unless contained is spreading like a wildfire (This is from the news as the analysis is yet to be done :D ).

I am also getting the total count of Confirmed patients,Patients who died and the recovered patients over the days.

Based on 3 columns i.e. confirmed,deaths and recovered cases we need to do comprehensive EDA with respect to -
    
    1. Date (Transition of date and the no. of cases )
    2. Province/State,Country (No. of cases based on country )

For this, if we create functions we can call all the three variables one by one and evaluate the results.

## Analysis based on Date

The approach I am planning is to first analyse the data based on date and how it has affected over a short span.I then plan to analyse it based on country .Cleaning up of data is needed as it is necessity to understand a little better.

In [ ]:
corona_data_date=pd.DataFrame(corona_data.groupby(by='Date').sum())
if 'SNo' in corona_data_date.columns:
    corona_data_date=corona_data_date.drop('SNo',axis=1)
corona_data_date['Date']=corona_data_date.index
corona_data_date.Date=corona_data_date.Date.apply(lambda x:x.date())

In [ ]:
melted_data=pd.melt(corona_data_date,id_vars=['Date'])

In [ ]:
def bar_plot(column_name):
    plt.figure(figsize=(10,15))
    plt.xticks(rotation=90)
    plt.xlabel('Date', fontsize=18)
    plot_1=sns.barplot(x='Date',y=column_name,data=corona_data_date)
    plot_1

In [ ]:
bar_plot('Recovered')

Initial graphs show that the recovery rate over these days has increased quite good. China especially created a hospital to treat and control the Corona virus within **10 Days** which is pretty amazing !!!

In [ ]:
bar_plot('Deaths')

As compared to Recovery , I checked on the no. of deaths and over these days it has a fair share of deaths.One thing one can do is check the no. of deaths to the no. of recovery since using that we can analyse how severity the virus actually is.

In [ ]:
bar_plot('Confirmed')

I think the above graph sums up all the analysis one needs to do.The virus has grown **EXPONENTIALLY** . This is because it spreads so quickly when people are talking or even sneezing. 

In [ ]:
hm=sns.catplot(x='Date', y='value', hue='variable', data=melted_data, kind='bar',height=10,aspect =1.6,legend=True)
hm.set_xticklabels( rotation=90)


As I was talking before, although the confirmed cases is large the virus has a higher recovery rate than the deaths but we can't be quite sure unless one does a statistical inference to prove this hypotheses.

In [ ]:
import plotly.express as px
fig = px.line(melted_data, x="Date",y='value', color='variable')
fig.show()

I love Plotly but never used it properly until now. It is quite interactive and presentable in nature. I need to learn a lot about plotly but initally I think plotly is quite easy to build

## Analysis based on Country 

I then concentrated on how the analysis worked based on the country. Clearly China is the epicentre of the virus.But it is also quite interesting to see how the virus spread to other countries as well !!!

    Confirmed - Cumulative number of confirmed cases till that date
    Deaths - Cumulative number of of deaths till that date
    Recovered - Cumulative number of recovered cases till that date

In [ ]:
#Since this is a cumulative dataset we can take the maximum of the dataset and then perform our analysis
latest_data_corona=corona_data[(corona_data.Date==max(corona_data["Date"]))]

In [ ]:
corona_data_country=pd.DataFrame(latest_data_corona.groupby(by='Country').sum())
if 'SNo' in corona_data_country:
    corona_data_country=corona_data_country.drop('SNo',axis=1)
corona_data_country['country']=corona_data_country.index

In [ ]:
print('Total no. of confirmed cases over these days',sum(latest_data_corona['Confirmed']))
print('Total no. of deaths over these days',sum(latest_data_corona['Deaths']))
print('Total no. of recovered cases over these days',sum(latest_data_corona['Recovered']))

In [ ]:
corona_data_country.sort_values(['Confirmed','Deaths','Recovered'],ascending=[False,False,False])

In [ ]:
corona_data_country["country"].replace({"Ivory Coast": "Cote d'Ivoire", 
                                        "Mainland China": "China",
                                        "Hong Kong":"Hong Kong, China",
                                       "South Korea":"Korea, Rep.",
                                        "UK":"United Kingdom",
                                        "US":"United States",
                                        "Macau" :"China"
                                       }, inplace=True)


I haven't worked on scattergeo of plotly and I couldn't find proper documents on how I can use plotly easily.I therefore did a workaround in which I found a work around to plot the values on to the maps.

In [ ]:
df = px.data.gapminder().query("year == 2007")
corona_data_country_geo_cd=pd.merge(corona_data_country,df,how='left',on='country')

In [ ]:
#Removing NaN based rows 
corona_data_country_geo_cd=corona_data_country_geo_cd.dropna(how='any')
corona_data_country_geo_cd=corona_data_country_geo_cd.drop(['year','lifeExp','pop','gdpPercap'],axis=1)

In [ ]:
fig = px.scatter_geo(corona_data_country_geo_cd[corona_data_country_geo_cd['country']!='China'], locations="iso_alpha",
                     size="Confirmed", # size of markers, "pop" is one of the columns of gapminder)
                    )
fig.show()

In [ ]:
fig = px.choropleth(corona_data_country_geo_cd[corona_data_country_geo_cd['country']!='China'], locations="iso_alpha",
                    color="Confirmed", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Electric)
fig.show()

In [ ]:
fig = px.choropleth(corona_data_country_geo_cd[corona_data_country_geo_cd['country']!='China'], locations="iso_alpha",
                    color="Deaths", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Oranges_r)
fig.show()

In [ ]:
fig = px.choropleth(corona_data_country_geo_cd[corona_data_country_geo_cd['country']!='China'], locations="iso_alpha",
                    color="Recovered", # lifeExp is a column of gapminder
                    hover_name="country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Oranges_r)
fig.show()

I plan to do further analysis based on China Region and then on how it is going to go further based on the regions over the time.

Please let me know If I need to make any changes. I am happy to learn. :)

In [ ]:
#Analysis of China Region
#corona_data.head()
corona_data["Country"].replace({"Ivory Coast": "Cote d'Ivoire", 
                                        "Mainland China": "China",
                                        "Hong Kong":"Hong Kong, China",
                                       "South Korea":"Korea, Rep.",
                                        "UK":"United Kingdom",
                                        "US":"United States",
                                        "Macau" :"China"
                                       }, inplace=True)
corona_data.Date=corona_data.Date.apply(lambda x:x.date())

In [ ]:
#Selecting data for only China regions including their province
China_data=corona_data[corona_data['Country']=='China']
print(China_data.columns)

In [ ]:
sns.catplot(y="Province/State", x="Confirmed", data=China_data,kind='boxen',height=9,aspect=1.4);


It is quite clear from the above graph is **Hubei** is the epicentre of the disease.It then spread onto other areas like **Anhui,Guangdong,Hunan,Henan**. We can confirm this when we look at those maps and then understand the flow of the virus.

In [ ]:
sns.catplot(y="Province/State", x="Deaths", data=China_data,kind='boxen',height=9,aspect=1.4);


The Death toll rate reached an all time high in Hubei region while others getting aware of the issue decided to make this issue a critical one.We can confirm this by checking the Recovered no. of patients.

In [ ]:
sns.catplot(y="Province/State", x="Recovered", data=China_data,kind='boxen',height=9,aspect=1.4);


Amazingly enough the regions which were getting the highest no. of Confirmed cases do have a higher rate of Recovery cases as well.

### Important Docs

1. [Time series](https://plot.ly/python/time-series/)